In [376]:
import json
import numpy as np
import pandas as pd

In [442]:
#importing the file that Guillaume sent us (data1)
with open ('./Hololens_data/p1.json') as json_file:
    data = json.load(json_file)

In [459]:
def norm(vect):
    sum = 0
    
    for el in vect:
        sum += el**2
    
    return np.sqrt(sum)

Let's create a Panda's dataFrame with position, time, rotatio, BPM  of each frame of the game and a second Dataframe with the balloons gathering data

In [548]:
def create_df_hand(game_file):
    with open(game_file) as json_file:
        data = json.load(json_file)
        
    df_game = pd.DataFrame(data['datasList'][0]['listLevelDatas'][0]['userDatas'])
    for i in range(1,len(data['datasList'][0]['listLevelDatas'])):
        df_game = pd.concat([df_game, pd.DataFrame(data['datasList'][0]['listLevelDatas'][i]['userDatas'])])
        
    
    
    #getting rid of the timeStamp's zero
    df_game = df_game[df_game['timeStamp']>0]
    
    #reset index after having got rid of the timeStamp zeros
    df_game = df_game.reset_index(drop = True) 
    
    #let's create three new columns, each one with one coordinate for df_game:
    #If they show later to be useless, we supprime these lines to get rid of them
    position =  df_game['headPos'].apply(pd.Series)
    df_game = pd.concat([df_game, position], axis=1)
    
    #Here we create a column withe a 4-element tuple: (x,y,z,t) for each dataframe
    df_game['head_positions'] = df_game[['x', 'y', 'z', 'timeStamp']].apply(lambda x: tuple(x), axis=1)
    
    for index, row in df_game.iterrows():
        if index != 0:
            if df_game.loc[index, 'timeStamp'] < df_game.loc[index-1, 'timeStamp']:
                #print('SHIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIT')
                for idx in range(index,len(df_game)):
                    df_game.loc[idx, 'timeStamp'] += df_game.loc[index-1, 'timeStamp']

    
    
    return df_game

In [549]:
def create_df_balloon(game_file):
        
    with open(game_file) as json_file:
        data = json.load(json_file)
        
        
    df_balloon = pd.DataFrame(data['datasList'][0]['listLevelDatas'][0]['listBalloonDatas'])
    for i in range(1,len(data['datasList'][0]['listLevelDatas'])):
        df_balloon = pd.concat([df_balloon, pd.DataFrame(data['datasList'][0]['listLevelDatas'][i]['listBalloonDatas'])])
    df_balloon = df_balloon.reset_index(drop = True)
    
    #this part of the code fixes the bug in the timeOfDestroy and timeOfSpawn that came with Hololens Data (values were reseting)
    for index, row in df_balloon.iterrows():
        if index != 0:
            if df_balloon.loc[index, 'timeOfDestroy'] < df_balloon.loc[index-1, 'timeOfDestroy']:
                for idx in range(index,len(df_balloon)):
                    df_balloon.loc[idx, 'timeOfDestroy'] += df_balloon.loc[index-1, 'timeOfDestroy']
                    df_balloon.loc[idx, 'timeOfSpawn'] += df_balloon.loc[index-1, 'timeOfSpawn']
    
    
    return df_balloon

In [560]:
df_balloon = create_df_balloon('./Hololens_data/p6.json')

In [561]:
df_game = create_df_hand('./Hololens_data/p6.json')

In [543]:
df_game.to_csv('tst.csv')

In [532]:
#import matplotlib as mp
#mp.histogram(np.diff(df_game['timeStamp']))

* The function `hand_positions` extracts the positions of the right hand along with the time corresponding to those positions. It returns an array of shape [(x, y, z, t)] (length number_of_position, with 4 elements arrays representing (x, y, z, t)).

In [562]:
def hand_positions(game_file):
    return list(create_df_hand(game_file)['head_positions'])
    

In [564]:
print(hand_positions('./Hololens_data/p6.json'))

[(0.015357498079538345, 0.08845899999141693, 0.20791688561439514, 0.03265779837965965), (0.03077283874154091, 0.08836336433887482, 0.2114529311656952, 0.21853360533714294), (0.03533522039651871, 0.08898738026618958, 0.21547836065292358, 0.28418004512786865), (0.03560098260641098, 0.08826440572738647, 0.22111302614212036, 0.35023510456085205), (0.0373358316719532, 0.08696436136960983, 0.22600123286247253, 0.41780850291252136), (0.040575772523880005, 0.08507534861564636, 0.23195433616638184, 0.48369619250297546), (0.040984004735946655, 0.08045218139886856, 0.23970657587051392, 0.5512577891349792), (0.03584291785955429, 0.07613646984100342, 0.24726825952529907, 0.6171257495880127), (0.025306925177574158, 0.074567511677742, 0.25342369079589844, 0.6837214231491089), (0.011081909760832787, 0.07364119589328766, 0.25566571950912476, 0.7507946491241455), (-0.0020746197551488876, 0.07743169367313385, 0.2570725679397583, 0.8184202313423157), (-0.00847998633980751, 0.07640080153942108, 0.260440617

In [565]:
traj = hand_positions('./Hololens_data/p8.json')
orientation(traj[0][0], traj[1][0], traj[0][1], traj[1][1], traj[0][2], traj[1][2])

[2.005411997912618, 163.55784622228168]

In [566]:
hand_positions('./Hololens_data/p6.json')

[(0.015357498079538345,
  0.08845899999141693,
  0.20791688561439514,
  0.03265779837965965),
 (0.03077283874154091,
  0.08836336433887482,
  0.2114529311656952,
  0.21853360533714294),
 (0.03533522039651871,
  0.08898738026618958,
  0.21547836065292358,
  0.28418004512786865),
 (0.03560098260641098,
  0.08826440572738647,
  0.22111302614212036,
  0.35023510456085205),
 (0.0373358316719532,
  0.08696436136960983,
  0.22600123286247253,
  0.41780850291252136),
 (0.040575772523880005,
  0.08507534861564636,
  0.23195433616638184,
  0.48369619250297546),
 (0.040984004735946655,
  0.08045218139886856,
  0.23970657587051392,
  0.5512577891349792),
 (0.03584291785955429,
  0.07613646984100342,
  0.24726825952529907,
  0.6171257495880127),
 (0.025306925177574158,
  0.074567511677742,
  0.25342369079589844,
  0.6837214231491089),
 (0.011081909760832787,
  0.07364119589328766,
  0.25566571950912476,
  0.7507946491241455),
 (-0.0020746197551488876,
  0.07743169367313385,
  0.2570725679397583,
  

* The function `bubble_pop` extracts the time of each game event corresponding to the pop of a bubble by the player. It returns an array of shape [t] (length number_of_bubble_poped).

In [567]:
def bubble_pop(game_file):
    return list(create_df_balloon(game_file)['timeOfDestroy'])


In [568]:
bubble_pop('./Hololens_data/p6.json')

[3.0986335277557373,
 10.924214363098145,
 15.886707305908203,
 21.64924430847168,
 27.26140594482422,
 29.243523955345154,
 37.702956199645996,
 40.18439960479736,
 45.09731101989746,
 48.794198989868164,
 78.73666858673096,
 81.71834993362427,
 84.9317741394043,
 87.54780578613281,
 90.94318103790283,
 171.4290051460266,
 171.4290051460266,
 175.22624683380127,
 181.1878318786621,
 181.1878318786621,
 184.8010835647583,
 184.8010835647583,
 188.64871501922607,
 188.64871501922607]

# Extraction of sub-trajectories & features
The function `sub_trajectories` returns an array of shape [[*[(x,y,t),(x,y,t),...]*, for each bubble in wave], for each wave]. To access all positions and time of the trajectory between the *i* and *i+1* bubble of the *n* wave : *sub_trajectories[n-1][i]*.

In [569]:
def sub_trajectories(game_file):
    hand_position = hand_positions(game_file)
    #bubble_pop_time = bubble_pop_clean(game_file)
    bubble_pop_time = bubble_pop(game_file)
    
    th = hand_position[0][3] #change to 3 because we have one more dimension
    
    sub_traj=[]
    
    nb_waves = len(bubble_pop_time)//5
    i=0 #loop count for waves
    k=0 #loop count for hand positions
    while i<nb_waves :
        sub_traj.append([])
        j=0 #loop count for bubbles
        while j<5:
            sub_traj[i].append([])
            t = bubble_pop_time[j+5*i] #the time the bubble was gathered
            while th < t:
                sub_traj[i][j].append(hand_position[k]) #appends the position of the hand and the corresponding time
                k+=1
                th = hand_position[k][3] #index out of range
            j+=1
        i+=1
    
    return np.array(sub_traj)

In [570]:
print(sub_trajectories('./Hololens_data/p6.json'))

IndexError: list index out of range

We define some functions to extract interesting features from trajectories. We first look for Static features : 
* `length` returns the length of the trajectory *traj*
* `barycenter` returns the barycenter of the trajectory *traj* in shape (x,y)
* `location` returns the average distance of each point to the barycenter of the trajectory *traj*
* `location_max` returns the maximum distance between a point of the trajectory and the barycenter of this trajectory
* `orientation` returns the angle between points the line between *(x1, y1)* and *(x2, y2)* and the horizontal axis (in degrees)
* `orientation_feat` returns the preceeding feature for the first two points and the last two points of the trajectory *traj*
* `nb_turns` returns the number of turns in the trajectory *traj*, where a turn is detected if the orientation between two consecutive couples of points varies of more than *limit_angle*

In [ ]:
def head_rotation(game_file):
    return list(create_df_hand(game_file)['headRotationY'])

In [ ]:
#head_rotation('./Hololens_data/p1.json')

In [473]:
def length(traj):
    l = 0
    
    for i in range(len(traj)-1):
        #l += np.sqrt((traj[i+1][0]-traj[i][0])**2 + (traj[i+1][1]-traj[i][1])**2) 
        l += np.sqrt((traj[i+1][0]-traj[i][0])**2 + (traj[i+1][1]-traj[i][1])**2+(traj[i+1][2]-traj[i][2])**2)
        
    
    return l



def barycenter(traj):
    x = 0
    y = 0
    z = 0
    n = len(traj)
    
    for i in range(n):
        x += traj[i][0]
        y += traj[i][1]
        z += traj[i][2]
    
    if n>0:
        return (x/n, y/n, z/n) #(x/n, y/n, z/n)
    else:
        return (0,0,0) #(0,0,0)

def location(traj):
    loc_avg = 0
    n = len(traj)
    p = barycenter(traj)
    
    for i in range(n):
        #loc_avg += np.sqrt((traj[i][0] - p[0])**2 + (traj[i][1] - p[1])**2) 
        loc_avg += np.sqrt((traj[i][0] - p[0])**2 + (traj[i][1] - p[1])**2+(traj[i][2]-p[2])**2)
        
    return loc_avg/n

def location_max(traj):
    n = len(traj)
    p = barycenter(traj)
    if n>0:
        l_max = np.max([np.sqrt((traj[i][0] - p[0])**2 + (traj[i][1] - p[1])**2+(traj[i][2]-p[2])**2) for i in range(n)])
        #l_max = np.max([np.sqrt((traj[i][0] - p[0])**2 + (traj[i][1] - p[1])**2) for i in range(n)]) 
        return l_max
    else:
        return 0

    
def orientation(x1, x2 , y1, y2, z1, z2):
    #if x2 == x1 and y2>=y1:
    #    return 90
    #elif x2 == x1 and y2<=y1:
    #    return -90
    #else:
    #    return np.arctan((y2 - y1)/(x2 - x1)) * (180/np.pi) #in degree
    if x2-x1<0:
        return [np.arctan((y2 - y1)/(x2 - x1)) * (180/np.pi),np.arctan((z2 - z1)/(x2 - x1)) * (180/np.pi)+180] #in degree
    elif z2-z1<0 and x2-x1>0:
        return [np.arctan((y2 - y1)/(x2 - x1)) * (180/np.pi),np.arctan((z2 - z1)/(x2 - x1)) * (180/np.pi)] #in degree
    if x2 == x1 and y2>=y1 and z2==z1:
        return [90,0]
    elif x2 == x1 and y2<=y1 and z2==z1:
        return [-90,0]
    elif x2-x1>0 and z2-z1>=0:
        return [np.arctan((y2 - y1)/(x2 - x1)) * (180/np.pi),np.arctan((z2 - z1)/(x2 - x1)) * (180/np.pi)+180] #in degree

def orientation_feat(traj):
    n = len(traj)
    if n>1:
        #ts = orientation(traj[0][0], traj[1][0], traj[0][1], traj[1][1])
        #te = orientation(traj[-2][0], traj[-1][0], traj[-2][1], traj[-1][1])
        ts = orientation(traj[0][0], traj[1][0], traj[0][1], traj[1][1], traj[0][2], traj[1][2])
        te = orientation(traj[-2][0], traj[-1][0], traj[-2][1], traj[-1][1], traj[-2][2], traj[-1][2])

        return (ts, te)
    else:
        #return (0,0)
        return ([0,0],[0,0])

def nb_turns(traj, limit_angle):
    nb_turns = 0
    n=len(traj)
    
    for i in range(n-2):
        if(np.abs(orientation(traj[i][0], traj[i+1][0], traj[i][1], traj[i+1][1], traj[i][2], traj[i+1][2])[0] - orientation(traj[i+1][0], traj[i+2][0], traj[i+1][1], traj[i+2][1], traj[i+1][2], traj[i+2][2])[0]) > limit_angle):
            nb_turns += 1
        #if(np.abs(orientation(traj[i][0], traj[i+1][0], traj[i][1], traj[i+1][1]) - orientation(traj[i+1][0], traj[i+2][0], traj[i+1][1], traj[i+2][1])) > limit_angle):
        #    nb_turns += 1
    
    return nb_turns

We then define dynamic features:
* `velocity` returns the list of the point to point velocities over the whole trajectory *traj*
* `velocity_avg` returns the average velocity over the trajectory *traj*
* `velocity_max` returns the greatest velocity over the trajectory *traj*
* `velocity_min` returns the lowest velocity over the trajectory *traj*
* `nb_vmin` returns the number of local minimum of velocity
* `nb_vmax` returns the number of local maximum of velocity

In [474]:
def velocity(traj):
    velocity = []
    
    for i in range(len(traj) - 1):
        #v = norm(np.array(traj)[i+1][:2] - np.array(traj)[i][:2]) / (np.array(traj)[i+1][3] - np.array(traj)[i][3])
        v = norm(np.array(traj)[i+1][:3] - np.array(traj)[i][:3]) / (np.array(traj)[i+1][3] - np.array(traj)[i][3])
        velocity.append(v)
        
    return np.array(velocity)

def velocity_avg(traj):
    v_avg = 0
    n = len(traj)
    if n>1:
        v_list = velocity(traj)

        for i in range(n-1):
            v_avg += v_list[i]

        return v_avg/(n-1)
    else:
        return 0

def velocity_max(traj):
    if len(traj)>1:
        return np.max(velocity(traj))
    else:
        return 0

def velocity_min(traj):
    if len(traj)>1:
        return np.min(velocity(traj))
    else:
        return 0

def nb_vmin(traj):
    nb = 0
    v_list = velocity(traj)
    
    for i in range(1,len(v_list)-1):
        if v_list[i]<v_list[i+1] and v_list[i]<v_list[i-1]:
            nb += 1
    
    return nb

def nb_vmax(traj):
    nb = 0
    v_list = velocity(traj)
    
    for i in range(1,len(v_list)-1):
        if v_list[i]>v_list[i+1] and v_list[i]>v_list[i-1]:
            nb += 1
    
    return nb

The function `feature_vector` extracts features from the trajectory in argument *traj = [(x,y,z)]*

In [475]:
def bucketize_nb_turns(nb_turn):
    if nb_turn == 0:
        return [1, 0, 0, 0]
    elif nb_turn == 1:
        return [0, 1, 0, 0]
    elif nb_turn < 4: # 2 ou 3
        return [0, 0, 1, 0]
    else:
        return [0, 0, 0, 1] #4 ou plus

In [476]:
def bucketize_nb_v(nb_v):
    if nb_v < 2:
        return [1, 0, 0, 0]
    elif nb_v < 4: # 2 ou 3
        return [0, 1, 0, 0]
    elif nb_v < 6: # 4 ou 5
        return [0, 0, 1, 0]
    else:
        return [0, 0, 0, 1] # 6 ou plus

old - 2d
def feature_vector(traj, playerID, game_area, limit_angle=0.25):
    diag = np.sqrt(game_area[0]**2 + game_area[1]**2)
    
    feature_vector = [playerID]
    
    feature_vector.append(length(traj)/diag)
    
    bc = barycenter(traj)
    feature_vector.append(np.float64(0.5 + bc[0] / game_area[0])) # between 0 and 1
    feature_vector.append(np.float64(0.5 + bc[1] / game_area[1]))
    
    if location_max(traj) == 0:
        feature_vector.append(np.float64(0))
    else:
        feature_vector.append(location(traj)/location_max(traj))
    
    angles = 0.5 + np.array(orientation_feat(traj)) / 180 # between 0 and 1
    feature_vector.append(angles[0]) #first orientation of traj
    feature_vector.append(angles[1]) #last orientation of traj
    
    feature_vector.append(nb_turns(traj, limit_angle))
    
    feature_vector.append(velocity_avg(traj))
    
    feature_vector.append(velocity_min(traj))
    feature_vector.append(velocity_max(traj))
    
    feature_vector.append(nb_vmin(traj))
    feature_vector.append(nb_vmax(traj))
    
    return feature_vector

In [477]:
def feature_vector(traj, playerID, game_area, limit_angle=0.25,Listefeatures=["dist/diag","game area","barycenter distance","angles","nb turns","velocity average","velocity min","velocity max","number of mins","number of maxs"]):
    diag = np.sqrt(game_area[0]**2 + game_area[1]**2)
    #listetot=[]
    dist=0
    feature_vector = [playerID]
    #for i in range(len(traj)):
    #    listetot+=traj[i]
    
    #feature_vector.append(head_rotation())
    dist=length(traj)
    bc=barycenter(traj)
    if "dist/diag" in Listefeatures:
        feature_vector.append(dist/diag)
    if "game area" in Listefeatures:
        feature_vector.append(np.float64(0.5 + bc[0] / game_area[0])) # between 0 and 1
        feature_vector.append(np.float64(0.5 + bc[1] / game_area[1]))
    if location_max(traj) == 0 and "barycenter distance" in Listefeatures:
            feature_vector.append(np.float64(0))
    elif "barycenter distance" in Listefeatures:
            feature_vector.append(location(traj)/location_max(traj))
    angles = 0.5 + np.array(orientation_feat(traj)) / 180 # between 0 and 1
    if "angles" in Listefeatures:
        feature_vector.append(angles[0][0]) #first orientation of traj
        feature_vector.append(angles[0][1])
        feature_vector.append(angles[1][1])#last orientation of traj
        feature_vector.append(angles[1][1])
    if "nb turns" in Listefeatures:
        feature_vector.append(nb_turns(traj, limit_angle))
    if "velocity average" in Listefeatures:
        feature_vector.append(velocity_avg(traj))
    if "velocity min" in Listefeatures:
        feature_vector.append(velocity_min(traj))
    if "velocity max" in Listefeatures:
        feature_vector.append(velocity_max(traj))
    if "number of mins" in Listefeatures:
        feature_vector.append(nb_vmin(traj))
    if "number of maxs" in Listefeatures:
        feature_vector.append(nb_vmax(traj))
    
    return feature_vector

In [478]:
def feature_vector_bucket(traj, playerID, game_area, limit_angle=0.25):
    diag = np.sqrt(game_area[0]**2 + game_area[1]**2)
    
    feature_vector = [playerID]
    
    feature_vector.append(length(traj)/diag)
    
    bc = barycenter(traj)
    feature_vector.append(np.float64(0.5 + bc[0] / game_area[0])) # between 0 and 1
    feature_vector.append(np.float64(0.5 + bc[1] / game_area[1]))
    
    if location_max(traj) == 0:
        feature_vector.append(np.float64(0))
    else:
        feature_vector.append(location(traj)/location_max(traj))
    
    angles = 0.5 + np.array(orientation_feat(traj)) / 180 # between 0 and 1
    feature_vector.append(angles[0]) #first orientation of traj
    feature_vector.append(angles[1]) #last orientation of traj
    
    bucket = bucketize_nb_turns(nb_turns(traj, limit_angle))
    for i in bucket:
        feature_vector.append(i)
    
    v_max = velocity_max(traj)
    if v_max == 0:
        feature_vector.append(0)
        feature_vector.append(0)
        feature_vector.append(0)
    else:
        feature_vector.append(velocity_avg(traj) / v_max)

        feature_vector.append(velocity_min(traj) / v_max)
        feature_vector.append(v_max)
    
    bucket_min = bucketize_nb_v(nb_vmin(traj))
    bucket_max = bucketize_nb_v(nb_vmax(traj))
    for i in bucket_min:
        feature_vector.append(i)
    for j in bucket_max:
        feature_vector.append(j)
    
    return feature_vector

The function `feature_vectors_game` allows to create the feature vectors over all the trajectories between the gathering of two bubbles of one game. The returned array is an array of multiple 13x5 arrays (the five feature vectors, containing 13 features each, corresponding to the five trajectories of each wave).

In [479]:
import ntpath

def feature_vectors_game(game_file, game_area = [21,10]):
    trajectories = np.array(sub_trajectories(game_file))
    nb_waves = len(trajectories)
    playerID = ntpath.basename(game_file)[:-5] #gets the name of the file as the player identity
    vectors = [[]]
    for i in range(0,nb_waves):
        vectors.append([])
        for traj in trajectories[i]:
            vectors[i].append(feature_vector(traj, playerID, game_area))
    
    return np.array(vectors)

In [480]:
def simple_features_generator(game_list):
    features=[]
    labels=[]
    for file in game_list:
        for layer1 in feature_vectors_game(file):
            for layer2 in layer1:
                features.append(layer2[1:])
                labels.append(layer2[0])
    np.savetxt('features.csv', features, delimiter=",")
    np.savetxt('output.csv', labels,delimiter=",", fmt='%s')
    return #features, labels

The following functions provide different shapes for the feature vector. This way of creating the feature vector could be improved by using tensorflow and its feature vectors, instead of creating it "by hand".
* "concat" means all features are concatenated into one numpy vector for each sample
* "bucket" means it uses the bucketized version of the feature vector (for nb_turns, nb_vmin, nb_vmax)
* "hands"  means it uses the hand used to play as label instead of the player's ID

In [481]:
def feature_vectors_game_concat(game_file, game_area = [21,10]):
    trajectories = np.array(sub_trajectories(game_file))
    nb_waves = len(trajectories)
    playerID = 1 #int(parse_root(game_file)[2][0].text) ## CHANGE THIS PART #################################
    vectors = []
    
    for i in range(nb_waves):
        vectors.append([])
        for traj in trajectories[i]:
            vectors[i] = vectors[i] + list(feature_vector(traj, playerID, game_area)[1:])
        vectors[i].append(playerID) ## CHANGE THIS PART #################################
    
    return np.array(vectors)

In [482]:
def feature_vectors_bucket_game_concat(game_file, game_area = [21,10]):
    trajectories = np.array(sub_trajectories(game_file))
    nb_waves = len(trajectories)
    playerID = 1#int(parse_root(game_file)[2][0].text)
    vectors = []
    
    for i in range(nb_waves):
        vectors.append([])
        for traj in trajectories[i]:
            vectors[i] = vectors[i] + list(feature_vector_bucket(traj, playerID, game_area)[1:])
        vectors[i].append(playerID)
    
    return np.array(vectors)

In [483]:
def feature_vectors_bucket_game_concat_hands(game_file, game_area = [21,10]):
    trajectories = np.array(sub_trajectories(game_file))
    nb_waves = len(trajectories)
    if parse_root(game_file)[2][2].text == 'false':
        useRightHand = 0
    else:
        useRightHand = 1
    vectors = []
    
    for i in range(nb_waves):
        vectors.append([])
        for traj in trajectories[i]:
            vectors[i] = vectors[i] + list(feature_vector_bucket(traj, useRightHand, game_area)[1:])
        vectors[i].append(useRightHand)
    
    return np.array(vectors)

Finally we provide a function to get the agregation of all feature vectors over multiple game files, where *game_files* is the list of the names (String type) of all the game files to be considered.

In [484]:
def agregate_feature_vectors(game_files):
    vectors = []
    for file in game_files:
        vectors = vectors + list(feature_vectors_game_concat(file))
    
    return np.array(vectors)

In [485]:
def agregate_feature_vectors_bucket(game_files):
    vectors = []
    for file in game_files:
        vectors = vectors + list(feature_vectors_bucket_game_concat(file))
    
    return np.array(vectors)

In [486]:
def agregate_feature_vectors_bucket_hands(game_files):
    vectors = []
    for file in game_files:
        vectors = vectors + list(feature_vectors_bucket_game_concat_hands(file))
    
    return np.array(vectors)

# Export of the final data

In [487]:
def export_feature_vectors(vectors, name):
    np.savetxt(name, vectors, delimiter=",")

In [488]:
relative_path = 'C:/Users/menoci/Desktop/Studies/autisme et ML/Code+Data/xml_data/'
print(relative_path+'abc.xml')

C:/Users/menoci/Desktop/Studies/autisme et ML/Code+Data/xml_data/abc.xml


In [489]:
relative_path = './Hololens_data/'

game_files=[relative_path + 'p1.json',
           relative_path + 'p3.json',
           relative_path + 'p4.json',
           relative_path + 'p5.json',
           relative_path + 'p6.json',
           relative_path + 'p7.json',
           relative_path + 'p8.json',
           relative_path + 'p9.json',
           relative_path + 'p10.json'] #here we add all the files we want to train the algorithm with

#vect = agregate_feature_vectors_bucket(game_files)
#export_feature_vectors(vect, "kate_data.csv")

In [490]:
simple_features_generator(['./Hololens_data/p8.json'])

IndexError: list index out of range